#1 라이브러리 및 모듈


In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 16.9 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 22.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp38-cp38-linux_x86_64.whl size=688995 sha256=35a3ad63c8f6a80b1aeadfe6e459150dd6efaad873266e6bb7abccbb9bf483e9
  Stored in directory: /root/.cache/pip/wheels/b6/93/9d/2237550c409eb3ed725d6302b7897ddd9a037b40cef66dcd9c
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-e3owu4pj
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-e3owu4pj
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.9/881.9 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook


In [4]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup


In [5]:
#GPU사용
device = torch.device("cuda:0")

#BERT모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


#2 데이터셋 전처리

In [7]:
import pandas as pd
test=pd.read_csv("test.csv")
train=pd.read_csv("train.csv")
df=pd.read_csv("df_labelling.csv",encoding="utf-8-sig")
#긍정:0 // 부정:1

In [8]:
test.loc[(test['label'] == "긍정"),'label'] = 0 #긍정:0
test.loc[(test['label'] == "부정"),'label'] = 1 #부정:1

In [9]:
train.loc[(train['label'] == "긍정"),'label'] = 0 #긍정:0
train.loc[(train['label'] == "부정"),'label'] = 1 #부정:1

In [10]:
df

,name,review,star,review_num,ave_star,label
0,인크커피 가산 플래그십 스토어,대형 카페 그잡채빵들이 너무 맛있다 커피 독특한데 맛남,4.0,400,4.4,0
1,인크커피 가산 플래그십 스토어,이 가격에 이 용량에 샷 싸장님 맘이라지만 ㅎㅎㅎ 한번 가본걸로 이만,2.0,62,2.1,1
2,인크커피 가산 플래그십 스토어,트리보러 가는 곳넓은데 사람 많음빵은 다 맛있었는데 오히려 커피는 아쉬움,4.0,25,4.1,0
3,인크커피 가산 플래그십 스토어,동네라 자주 가는데 어쩔땐 친절하고 어쩔땐 불친절하고 알바바이알바인듯 ㅋㅋㅋ 근데 ...,4.0,90,3.4,0
4,인크커피 가산 플래그십 스토어,카페 분위기는 좋으나 디저트 준비하는 곳에서 일하시는 여자 알바가 너무 싸가지 없어...,1.0,1,1.0,1
...,...,...,...,...,...,...
33048,설빙 노량진점,노량진점 설빙 진짜별로네요 최악이에요 고객한테 대하는 전화 말투법이 상당히 거칠고 ...,1.0,2,1.0,1
33049,설빙 노량진점,배달 시켰는데 다른 지점보다 양이 많아서 놀람,5.0,29,4.2,0
33050,설빙 노량진점,맛잇음 다른 설빙 가게들보다 양도 많은거 같고,4.0,10,1.9,0
33051,설빙 노량진점,체육관 개와 포켓스탑 개 잡힘영등포 설빙은 시설도 빙수 퀄리티도 최악인데 여기는 ...,4.0,318,3.4,0


In [11]:
data_list=[]
for q, label in zip(df['review'],df['label']) :
  data = []
  data.append(q)
  data.append(str(label))

  data_list.append(data)

In [12]:
data_list

[['대형 카페 그잡채빵들이 너무 맛있다 커피 독특한데 맛남', '0'],
 ['이 가격에 이 용량에 샷 싸장님 맘이라지만 ㅎㅎㅎ 한번 가본걸로 이만', '1'],
 ['트리보러 가는 곳넓은데 사람 많음빵은 다 맛있었는데 오히려 커피는 아쉬움', '0'],
 ['동네라 자주 가는데 어쩔땐 친절하고 어쩔땐 불친절하고 알바바이알바인듯 ㅋㅋㅋ 근데 마감직전에 갔을땐 사장님인 것 같은데 친절하셨다 카페가 크고 디저트나 음료도 다양해서 딱히 갈데는 없는데 데이트 기분 내면서 밖에 있고 싶을땐 여기를 들리는 것 같다디저트랑 음료 모두 평타 저 로비앞에서 남자는 하나같이 여친 찍어주고 있고 ㅋㅋㅋ 나름 가산을 대표하는 카페인건 확실',
  '0'],
 ['카페 분위기는 좋으나 디저트 준비하는 곳에서 일하시는 여자 알바가 너무 싸가지 없어서 기분 다 망쳤습니다 인원이 많아 테이블 두개로 나눠 앉아서 두개의 트레이에 디저트를 나눠 담고 한번에 결제했는데주문서도 잘 안보고 준비하는지 트레이 둘 다 챙겨가려하니 그쪽거 아니라고 하길래 저희꺼 맞아요라고 하니 아니라고요라고 하며 못 가져가게 해서 저희가 시킨거 맞고 나눠서 담았다니까 주문서도 다시 찾아보지않는 완강한 태도와 싸가지없는 말투오로 그쪽 테이블거 아니라고요하며 무시하듯 답변몇번을 맞다 아니다로 할게 아니라 이쯤되면 주문서 한번 다시 볼법도 하지않나요ㅎㅎ어찌나 황당하던지버터바 두개시켰는데 하나밖에 없고 저희가 시킨게 맞다니까 그제서야 주문서 보더니 다른 테이블 다른 디저트 주문서를 밑에 끼워두고는 이게 뭐야라며 당황하시더라고요끝까지 단 한마디의 사과도 받진 못했습니다몇번을 맞다고 하면 적어도 주문서를 다시 한번 보실 생각은 없으신가요 무시하는 듯한 말투 째려보는 눈 뭐 엄청난 친절과 서비스를 바라며 카페를 가진 않으나 아무리 그래도 본인이 착각하신거면 적어도 사과는 하셨어야하는 거 아닌가 싶습니다',
  '1'],
 ['빵 맛있음', '0'],
 ['트리가 너모 예뻐요ㅠㅠㅜㅜ사실 트리만 보고 방문했는데커피랑 빵도 맛있고카페

#3 KoBERT 입력 데이터로 만들기


In [13]:
train_list=[]
for q,label in zip(train['review'],train['label']) :
  data =[]
  data.append(q)
  data.append(str(label))

  train_list.append(data)

In [14]:
test_list=[]
for q,label in zip(test['review'],test['label']) :
  data =[]
  data.append(q)
  data.append(str(label))

  test_list.append(data)

In [15]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [16]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [17]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [18]:
data_train = BERTDataset(train_list, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test_list,0, 1, tok, max_len, True, False)

In [19]:
data_train[0]

(array([   2, 3301, 3298, 6493, 7178, 6424, 7489, 7217, 6896, 4162, 7903,
        6361, 7946, 7172, 7292, 3758, 6573,    0,  490, 6896, 1282, 1370,
           0, 5761, 3327, 5655, 6944, 5496,  492,  492,  492,  862, 6113,
        5760, 4641, 4081, 6881, 6855, 6553, 3093, 6657, 5782,    3,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(43, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [20]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


#4 KoBERT 학습모델 만들기


In [21]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [22]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

#5 KoBERT 모델 학습시키기

In [23]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-23-480b6a139979>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/414 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.841899573802948 train acc 0.296875
epoch 1 batch id 201 loss 0.22375646233558655 train acc 0.7897232587064676
epoch 1 batch id 401 loss 0.1704305112361908 train acc 0.8448410224438903
epoch 1 train acc 0.8467919685990337


<ipython-input-23-480b6a139979>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/104 [00:00<?, ?it/s]

...

epoch 9 test acc 0.9374446482793523


  0%|          | 0/414 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.0012090300442650914 train acc 1.0
epoch 10 batch id 201 loss 0.0023822777438908815 train acc 0.9959577114427861
epoch 10 batch id 401 loss 0.00106522545684129 train acc 0.9964541770573566
epoch 10 train acc 0.9963013285024155


  0%|          | 0/104 [00:00<?, ?it/s]

epoch 10 test acc 0.9381958502024291


#6 새로운 문장 테스트


In [24]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("긍정적인")
            elif np.argmax(logits) == 1:
                test_eval.append("부정적인")

        print(">> 입력하신 내용은 " + test_eval[0] + "리뷰입니다.")

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [25]:
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 맛있고 직원이 친절해요
>> 입력하신 내용은 긍정적인리뷰입니다.


하고싶은 말을 입력해주세요 : 그냥 그래요
>> 입력하신 내용은 부정적인리뷰입니다.


하고싶은 말을 입력해주세요 : 맛있지는 않아요
>> 입력하신 내용은 부정적인리뷰입니다.


하고싶은 말을 입력해주세요 : 추천을 안하지는 않을 듯
>> 입력하신 내용은 부정적인리뷰입니다.


하고싶은 말을 입력해주세요 : 0
>> 입력하신 내용은 긍정적인리뷰입니다.


하고싶은 말을 입력해주세요 : 1
>> 입력하신 내용은 긍정적인리뷰입니다.


하고싶은 말을 입력해주세요 : 2
>> 입력하신 내용은 부정적인리뷰입니다.


하고싶은 말을 입력해주세요 : 3
>> 입력하신 내용은 부정적인리뷰입니다.


하고싶은 말을 입력해주세요 : 0
>> 입력하신 내용은 긍정적인리뷰입니다.


하고싶은 말을 입력해주세요 : !0
>> 입력하신 내용은 긍정적인리뷰입니다.




KeyboardInterrupt: ignored